# Les algorithmes gloutons

## Problème du rendu de monnaie

### Rendu de monnaie en euros

On travaille dans un premier temps avec les pièces/billets disponibles en euros:

In [1]:
# On ne fait pas la distinction artificielle entre pièces et billets
types_de_pièces_euro = [1, 2, 5, 10, 20, 50, 100, 200]

#### Exercice 1 (programmation)

Pour ce premier exercice, on écrit un algorithme le plus simple possible: il ne fonctionne que pour les euros, par pour un système exotique. On examinera la version plus complète un peu plus tard.

Notons qu'on utilise ici la fonction ```format``` afin de créer facilement des chaînes de caractères contenant des valeurs extraites de variables. Chaque double accolades ```{}``` sera remplacée par la valeur correspondante passée en paramètre à ```format```.

In [2]:
def rendre_monnaie(somme):
    somme_initiale = somme
    print("On veut rendre {}€.".format(somme_initiale))     # Le {} sera remplacé par la valeur de somme
    
    pièces = []
    while somme > 0:
        if somme >= 200:
            rendre = 200
        elif somme >= 100:
            rendre = 100
        elif somme >= 50:
            rendre = 50
        elif somme >= 20:
            rendre = 20
        elif somme >= 10:
            rendre = 10
        elif somme >= 5:
            rendre = 5
        elif somme >= 2:
            rendre = 2
        else:
            rendre = 1
        
        pièces.append(rendre)
        print("On rend une pièce de {}€".format(rendre))
        somme = somme - rendre
        
    print("Vérification: {} = {}".format(somme_initiale, "+".join([str(p) for p in pièces])))

In [3]:
rendre_monnaie(483)

On veut rendre 483€.
On rend une pièce de 200€
On rend une pièce de 200€
On rend une pièce de 50€
On rend une pièce de 20€
On rend une pièce de 10€
On rend une pièce de 2€
On rend une pièce de 1€
Vérification: 483 = 200+200+50+20+10+2+1


#### Exercice 2 (programmation)

Modifier la fonction précédente de deux manières:
* Sa signature sera ```rendre_monnaie(somme, types_pièces)```, ou le second paramètre est le tableau listant les types de pièces disponibles. On supposera que ce tableau est toujours donné dans l'ordre croissant.
* Cette fonction n'affichera absolument rien dans la consolle, mais renverra deux valeurs (sous la forme d'un tuple):
  * un dictionnaire donnant le nombre de pièces à rendre, pour chaque type.
  * la somme restant à rendre à la fin de l'algorithme. Elle sera nulle si l'algorithme glouton a complètement abouti, mais pourra être stritement positive en cas d'échec (impossible avec le système Européen).

Voici un exemple:
```python
>>> rendre_monnaie(83, types_pièces_euro)
({ 1: 1, 2: 1, 5: 0, 10: 1, 20: 1, 50: 1, 100: 0, 200: 0}, 0)
```

Avec d'autres systèmes que la monnaie Européenne, il se pourrait que l'algorithme glouton ne donne pas la solution optimale:

```python
>>> rendre_monnaie(12, [1, 6, 10])
({ 1: 2, 6: 0, 10: 1}, 0)
```
L'algorithme a donc donné $12 = 10 + 1 + 1$, alors que la solution optimale est $12 = 6 + 6$.

On peut aussi ne pas avoir de solution du tout:
```python
>>> rendre_monnaie(4, [2, 3])
({ 2: 0, 3: 1}, 1)
```

L'algorithme a donc choisi la pièce de 3 et il ne peut pas rendre la monnaie pour le reste de 1, alors que la solution $4 = 2 + 2$ existe et est optimale, mais pas découvrable par un algorithme glouton.

In [5]:
def rendre_monnaie(somme, type_pièces):
    dico = {}
    
    # L'indice de la plus grande pièce est égal à la longueur du tableau moins un:
    indice_pièce = len(type_pièces) - 1
    
    # La condition d'arrêt est un peu plus compliquée: on s'arrête bien évidemment si la
    # somme restant à rendre devient nulle. Mais on doit aussi s'arrêter si l'indice_pièce
    # devient strictement négatif: cela signifie alors qu'il n'y a pas de pièces assez petites
    # pour rendre la monnaie restante (par exemple il reste 1€ à rendre mais la plus petite
    # pièce vaut 2€).
    #
    while somme > 0 and indice_pièce >= 0:
        valeur_pièce = type_pièces[indice_pièce]
        if somme >= valeur_pièce:
            # On a trouvé une pièce assez grande, on la rend...
            somme = somme - valeur_pièce
            
            # ... et on la rajoute au dictionnaire. On commence par regarder
            # si on avait déjà rendu cette pièce auparavant:
            if valeur_pièce in dico:
                # On avait déjà rendu au moins une fois ce type de pièce:
                # on en rend une de plus
                dico[valeur_pièce] = dico[valeur_pièce] + 1
            else:
                # C'est la première fois que l'on rend ce type de pièce
                dico[valeur_pièce] = 1
        else:
            # La pièce actuelle est trop grande: on examine la suivante (s'il en reste...)
            indice_pièce = indice_pièce - 1

    return (dico, somme)

In [7]:
rendre_monnaie(483, types_de_pièces_euro)

({200: 2, 50: 1, 20: 1, 10: 1, 2: 1, 1: 1}, 0)

Testons les cas un peu plus exotiques indiqués dans l'énoncé:

In [9]:
rendre_monnaie(12, [1, 6, 10])

({10: 1, 1: 2}, 0)

In [11]:
rendre_monnaie(4, [2, 3])

({3: 1}, 1)

On se rend compte que l'algorithme glouton ne fournit pas de solution complète dans ce dernier cas, bien qu'une solution existe.

#### Exercice 3 (programmation)

Utiliser la somme précédente pour calculer le nombre maximum de pièces/billets à utiliser en Europe lorsque l'on souhaite rendre une somme entière strictement inférieure à 500€.

Pour quelle somme ce nombre maximal est-il atteint ?

On va procéder de manière très simple:
  * On appelle la fonction précédente sur toutes les valeurs en euros jusqu'à 499€. 
  * Grâce au dictionnaire, on compte le nombre de pièces utilisées
  * On cherche le nombre de pièces maximal grâce à l'algorithme maintenant bien connu de recherche de maximum. Comme il peut y avoir plusieurs valeurs requierant le même nombre de pièces, on les stocke toutes dans une liste.

In [15]:
# Le nombre maximal de pièce trouvé jusqu'à présent. On peut initialiser raisonnablement à 0
nmax = 0

# Un tableau stockant toutes les valeurs requierant un nombre maximal de pièces.
valeurs_maximales = []

for valeur in range(1, 500):
    dico, restant = rendre_monnaie(valeur, types_de_pièces_euro)
    # Les mathématiques nous disent que l'algorithme glouton 
    # DOIT fonctionner pour les euros.
    assert restant == 0 
    
    nombre = 0
    for pièce in dico:
        # On rajoute le nombre de fois que la pièce de valeur 'pièce' a été utilisée:
        nombre = nombre + dico[pièce]
        
    if nombre == nmax:
        # On a une autre valeur ex-aequo:
        valeurs_maximales.append(str(valeur))
    elif nombre > nmax:
        # On a un nouveau maximum: il faut réinitialiser le tableau des valeurs
        # déjà trouvées:
        nmax = nombre
        valeurs_maximales = []
        
print("Il faut rendre au maximum {} pièces, pour les valeurs {}.".format(nmax, ", ".join(valeurs_maximales)))


Il faut rendre au maximum 8 pièces, pour les valeurs 389, 398, 399, 488, 489, 498, 499.


## Problème du voyageur de commerce

## Problème du sac à dos

Voici quelques situations:

* **Situation 1**
  * **Objet A**: 8kg / 4800€
  * **Objet B**: 5kg / 4000€
  * **Objet C**: 4kg / 3000€
  * **Objet D**: 1kg / 500€
* **Situation 2**
  * **Objet A**: 6kg / 4800€
  * **Objet B**: 5kg / 3500€
  * **Objet C**: 4kg / 3000€
  * **Objet D**: 1kg / 500€
* **Situation 3**
  * **Objet A**: 9kg / 8100€
  * **Objet B**: 6kg / 7200€
  * **Objet C**: 5kg / 5500€
  * **Objet D**: 4kg / 4000€
  * **Objet E**: 1kg / 800€
* **Situation 4**
  * **Objet A**: 7kg / 9100€
  * **Objet B**: 6kg / 7200€
  * **Objet C**: 4kg / 4800€
  * **Objet D**: 3kg / 2700€
  * **Objet E**: 2kg / 2600€
  * **Objet F**: 1kg / 200€
  


On commence par saisir les différentes valeurs dans des structures python:

In [16]:
situation1 = { "A": (8, 4800), "B": (5, 4000), "C": (4, 3000), "D": (1, 500)}
situation2 = { "A": (6, 4800), "B": (5, 3500), "C": (4, 3000), "D": (1, 500)}
situation3 = { "A": (9, 8100), "B": (6, 7200), "C": (5, 5500), "D": (4, 4000), "E": (1, 800)}
situation4 = { "A": (7, 9100), "B": (6, 7200), "C": (4, 4800), "D": (3, 2700), "E": (2, 2600), "F": (1, 200)}

#### Exercice 4 (à la main)

1. Pour chacune de ses situations, déterminer les différentes combinaisons possibles pour rendre le sac, ainsi que la valeur correspondante.

On propose ici deux fonctions qui recherchent **toutes** les combinaisons donnant exactement le poids demandé, et les affichant des plus chères aux moins chères.

**Attention:** L'algorithme utilisé ici est *très* complexe, n'essayez surtout pas de comprendre ce code (mais je ne vais pas non plus vous l'interdire). C'est largement au-delà de ce qu'on peut raisonnablement attendre d'un élève en NSI, même si vous étiez en Terminale. Je reste cependant ouvert à toutes les questions, je ne veux bloquer personne dans sa soif de connaissances.

In [76]:
def combinaisons(situation, poids_max):
    resultats = [[] for _ in range(poids_max + 1)]
    
    for objet in situation:
        poids, valeur = situation[objet]
        resultats[poids].append({objet: 1})
    
    for p in range(1, poids_max + 1):
        for objet in situation:
            poids, valeur = situation[objet]
            if p - poids > 0:
                for dico in resultats[p - poids]:
                    
                    copie = {}
                    for q in dico:
                        copie[q] = dico[q]
                        
                    if objet in copie:
                        copie[objet] = copie[objet] + 1
                    else:
                        copie[objet] = 1
                    resultats[p].append(copie)
                    
    return resultats[poids_max]

In [77]:
def affiche_combinaisons(situation, poids_total):
    resultats = []
    for dico in combinaisons(situation, poids_total):
        total = 0
        poids = 0
        chaîne = []
        for objet in dico:
            chaîne.append("{}x{}({}kg)".format(dico[objet], objet, situation[objet][0]))
            total = total + situation[objet][1]*dico[objet]
            poids = poids + situation[objet][0]*dico[objet]
        assert poids == poids_total
        resultats.append((-total, ", ".join(chaîne) + " -> {}€".format(total)))
    resultats.sort()
    for _, chaîne in resultats:
        print(chaîne)


In [78]:
affiche_combinaisons(situation1, 10)

2xB(5kg) -> 8000€
1xB(5kg), 1xC(4kg), 1xD(1kg) -> 7500€
1xB(5kg), 1xD(1kg), 1xC(4kg) -> 7500€
1xC(4kg), 1xB(5kg), 1xD(1kg) -> 7500€
1xC(4kg), 1xD(1kg), 1xB(5kg) -> 7500€
1xD(1kg), 1xB(5kg), 1xC(4kg) -> 7500€
1xD(1kg), 1xC(4kg), 1xB(5kg) -> 7500€
2xC(4kg), 2xD(1kg) -> 7000€
2xC(4kg), 2xD(1kg) -> 7000€
2xC(4kg), 2xD(1kg) -> 7000€
2xD(1kg), 2xC(4kg) -> 7000€
2xD(1kg), 2xC(4kg) -> 7000€
2xD(1kg), 2xC(4kg) -> 7000€
1xB(5kg), 5xD(1kg) -> 6500€
5xD(1kg), 1xB(5kg) -> 6500€
5xD(1kg), 1xB(5kg) -> 6500€
5xD(1kg), 1xB(5kg) -> 6500€
5xD(1kg), 1xB(5kg) -> 6500€
5xD(1kg), 1xB(5kg) -> 6500€
1xC(4kg), 6xD(1kg) -> 6000€
6xD(1kg), 1xC(4kg) -> 6000€
6xD(1kg), 1xC(4kg) -> 6000€
6xD(1kg), 1xC(4kg) -> 6000€
6xD(1kg), 1xC(4kg) -> 6000€
6xD(1kg), 1xC(4kg) -> 6000€
6xD(1kg), 1xC(4kg) -> 6000€
1xA(8kg), 2xD(1kg) -> 5800€
2xD(1kg), 1xA(8kg) -> 5800€
2xD(1kg), 1xA(8kg) -> 5800€
10xD(1kg) -> 5000€


In [79]:
affiche_combinaisons(situation2, 10)

1xA(6kg), 1xC(4kg) -> 7800€
1xC(4kg), 1xA(6kg) -> 7800€
1xB(5kg), 1xC(4kg), 1xD(1kg) -> 7000€
1xB(5kg), 1xD(1kg), 1xC(4kg) -> 7000€
1xC(4kg), 1xB(5kg), 1xD(1kg) -> 7000€
1xC(4kg), 1xD(1kg), 1xB(5kg) -> 7000€
1xD(1kg), 1xB(5kg), 1xC(4kg) -> 7000€
1xD(1kg), 1xC(4kg), 1xB(5kg) -> 7000€
2xB(5kg) -> 7000€
2xC(4kg), 2xD(1kg) -> 7000€
2xC(4kg), 2xD(1kg) -> 7000€
2xC(4kg), 2xD(1kg) -> 7000€
2xD(1kg), 2xC(4kg) -> 7000€
2xD(1kg), 2xC(4kg) -> 7000€
2xD(1kg), 2xC(4kg) -> 7000€
1xA(6kg), 4xD(1kg) -> 6800€
4xD(1kg), 1xA(6kg) -> 6800€
4xD(1kg), 1xA(6kg) -> 6800€
4xD(1kg), 1xA(6kg) -> 6800€
4xD(1kg), 1xA(6kg) -> 6800€
1xB(5kg), 5xD(1kg) -> 6000€
1xC(4kg), 6xD(1kg) -> 6000€
5xD(1kg), 1xB(5kg) -> 6000€
5xD(1kg), 1xB(5kg) -> 6000€
5xD(1kg), 1xB(5kg) -> 6000€
5xD(1kg), 1xB(5kg) -> 6000€
5xD(1kg), 1xB(5kg) -> 6000€
6xD(1kg), 1xC(4kg) -> 6000€
6xD(1kg), 1xC(4kg) -> 6000€
6xD(1kg), 1xC(4kg) -> 6000€
6xD(1kg), 1xC(4kg) -> 6000€
6xD(1kg), 1xC(4kg) -> 6000€
6xD(1kg), 1xC(4kg) -> 6000€
10xD(1kg) -> 5000€


In [80]:
affiche_combinaisons(situation3, 10)

1xB(6kg), 1xD(4kg) -> 11200€
1xD(4kg), 1xB(6kg) -> 11200€
2xC(5kg) -> 11000€
1xB(6kg), 4xE(1kg) -> 10400€
4xE(1kg), 1xB(6kg) -> 10400€
4xE(1kg), 1xB(6kg) -> 10400€
4xE(1kg), 1xB(6kg) -> 10400€
4xE(1kg), 1xB(6kg) -> 10400€
1xC(5kg), 1xD(4kg), 1xE(1kg) -> 10300€
1xC(5kg), 1xE(1kg), 1xD(4kg) -> 10300€
1xD(4kg), 1xC(5kg), 1xE(1kg) -> 10300€
1xD(4kg), 1xE(1kg), 1xC(5kg) -> 10300€
1xE(1kg), 1xC(5kg), 1xD(4kg) -> 10300€
1xE(1kg), 1xD(4kg), 1xC(5kg) -> 10300€
2xD(4kg), 2xE(1kg) -> 9600€
2xD(4kg), 2xE(1kg) -> 9600€
2xD(4kg), 2xE(1kg) -> 9600€
2xE(1kg), 2xD(4kg) -> 9600€
2xE(1kg), 2xD(4kg) -> 9600€
2xE(1kg), 2xD(4kg) -> 9600€
1xC(5kg), 5xE(1kg) -> 9500€
5xE(1kg), 1xC(5kg) -> 9500€
5xE(1kg), 1xC(5kg) -> 9500€
5xE(1kg), 1xC(5kg) -> 9500€
5xE(1kg), 1xC(5kg) -> 9500€
5xE(1kg), 1xC(5kg) -> 9500€
1xA(9kg), 1xE(1kg) -> 8900€
1xE(1kg), 1xA(9kg) -> 8900€
1xD(4kg), 6xE(1kg) -> 8800€
6xE(1kg), 1xD(4kg) -> 8800€
6xE(1kg), 1xD(4kg) -> 8800€
6xE(1kg), 1xD(4kg) -> 8800€
6xE(1kg), 1xD(4kg) -> 8800€
6xE(1kg), 1x

In [81]:
affiche_combinaisons(situation4, 10)

5xE(2kg) -> 13000€
1xC(4kg), 3xE(2kg) -> 12600€
3xE(2kg), 1xC(4kg) -> 12600€
3xE(2kg), 1xC(4kg) -> 12600€
3xE(2kg), 1xC(4kg) -> 12600€
1xB(6kg), 2xE(2kg) -> 12400€
2xE(2kg), 1xB(6kg) -> 12400€
2xE(2kg), 1xB(6kg) -> 12400€
1xE(2kg), 2xC(4kg) -> 12200€
2xC(4kg), 1xE(2kg) -> 12200€
2xC(4kg), 1xE(2kg) -> 12200€
1xB(6kg), 1xC(4kg) -> 12000€
1xC(4kg), 1xB(6kg) -> 12000€
1xA(7kg), 1xE(2kg), 1xF(1kg) -> 11900€
1xA(7kg), 1xF(1kg), 1xE(2kg) -> 11900€
1xE(2kg), 1xA(7kg), 1xF(1kg) -> 11900€
1xE(2kg), 1xF(1kg), 1xA(7kg) -> 11900€
1xF(1kg), 1xA(7kg), 1xE(2kg) -> 11900€
1xF(1kg), 1xE(2kg), 1xA(7kg) -> 11900€
1xA(7kg), 1xD(3kg) -> 11800€
1xD(3kg), 1xA(7kg) -> 11800€
2xF(1kg), 4xE(2kg) -> 10800€
2xF(1kg), 4xE(2kg) -> 10800€
2xF(1kg), 4xE(2kg) -> 10800€
2xF(1kg), 4xE(2kg) -> 10800€
2xF(1kg), 4xE(2kg) -> 10800€
4xE(2kg), 2xF(1kg) -> 10800€
4xE(2kg), 2xF(1kg) -> 10800€
4xE(2kg), 2xF(1kg) -> 10800€
4xE(2kg), 2xF(1kg) -> 10800€
4xE(2kg), 2xF(1kg) -> 10800€
4xE(2kg), 2xF(1kg) -> 10800€
4xE(2kg), 2xF(1kg) -> 

On peut constater que, suivant les situations, les combinaisons les plus intéressantes n'utilisent pas nécessairement les objets les plus lourds. 

La recherche d'un algorithme glouton ne va donc pas être aisée: doit-on privilégier les objets les plus lourds ? les moins lourds ? les plus ou moins chers ? un peu des deux ?

1. Arsène Lupin n'a que peu de temps pour choisir les objets à fourrer dans son sac. Il n'a pas matériellement le temps de tester toutes les combinaisons, et décide d'opter pour une stratégie gloutonne. Il envisage plusieurs possibilités:

  1. Choisir les objets par ordre de valeur décroissante parmi ceux qui ne dépassent pas la capacité restante.
  1. choisir les objets par ordre de poids croissant.
  1. choisir les objets par ordre de rapport valeur/poids décroissant parmi ceux qui ne dépassent pas la capacité restante.

  Appliquer chacune de ces 3 stratégies aux situations précédentes, comparer les résultats entre eux, et les comparer à la solution optimale trouvée par la recherche exhaustive précédente.

#### Exercice 5 (programmation)

La clé pour utiliser un algorithme glouton dans une des 4 situations précédentes est de réussir à trier les objets du plus intéressant au moins intéressant, la définition d'*intéressant* dépendant des choix proposés:
  * Dans le premier cas, on va trier les objets du plus lourd au moins lourd
  * Dans le deuxième cas, on va trier les objets du moins lourds au plus lourd
  * Dans le troisième cas, on va les trier par ordre descendant des rapports valeur/poids
  
On peut décomposer cet exercice de programmation en deux parties:
  * Écrire trois fonctions prenant pour paramètre une des 4 situations, et retournant un tableau de tuples au format ("Objet", poids, valeur) trié dans le bon ordre (voir ci-dessus)
  * Utiliser ces 3 fonctions pour implémenter un algorithme glouton. Remarquez qu'un seul algorithme glouton suffit pour les 3 stratégies, seul l'ordre du tri sera important.

En guise d'exemple, on donne la fonction de tri pour la première stratégie:

In [93]:
def clé_de_tri_A(clé):
    # On veut ici trier suivant l'ordre décroissant des poids
    #
    # On utilise l'astuce consistant à retourner des nombres négatifs afin d'inverser l'ordre
    # naturel du tri.
    #
    # La fonction sort de python fonctionne exactement de la même fonction que les fonctions
    # de tri que nous avons étudié il y a quelques temps.
    objet, poids, valeur = clé
    return -poids

def trie_objets(situation, clé_de_tri):
    # On commence par créer le tableau des tuples:
    tableau = []
    for objet in situation:
        poids, valeur = situation[objet]
        tableau.append((objet, poids, valeur))
        
    tableau.sort(key = clé_de_tri)
    return tableau

In [95]:
for situation in [situation1, situation2, situation3, situation4]:
    print(trie_objets(situation, clé_de_tri_A))

[('A', 8, 4800), ('B', 5, 4000), ('C', 4, 3000), ('D', 1, 500)]
[('A', 6, 4800), ('B', 5, 3500), ('C', 4, 3000), ('D', 1, 500)]
[('A', 9, 8100), ('B', 6, 7200), ('C', 5, 5500), ('D', 4, 4000), ('E', 1, 800)]
[('A', 7, 9100), ('B', 6, 7200), ('C', 4, 4800), ('D', 3, 2700), ('E', 2, 2600), ('F', 1, 200)]


On constate que les objets sont bien triés suivant l'ordre décroissant des poids.

Il ne reste pour notre algorithme glouton plus qu'à considérer les objets dans l'ordre dans lequel ils sont rangés dans le tableau.

Pour les autres stratégies, il vous suffit d'écrire les fonctions ```clé_de_tri_B``` et ```clé_de_tri_C```: les fonctions ```trie_objets``` et ```remplissage_glouton``` ne changent pas d'une stratégie à l'autre.

À vous de jouer !

N'oubliez pas de comparer les résultats des stratégies gloutones avec la recherche exhaustive affichée plus haut.

In [96]:
def remplissage_glouton(objets_triés, poids_max):
    pass

## Problème de l'emploi du temps

Supposons avoir une liste d'activités. Chacune de ces activités est associée à un créneau horaire défini par une heure de début et une heure de fin. 

Deux activités sont compatibles si leurs créneaux horaires ne se chevauchent pas.

On souhaite sélectionner un nombre maximal d'activités dans une journée, toutes compatibles entre elles.

#### Exercice 6 (à la main)

1. On se donne des activités dans les créneaux suivants: 8h-13h, 12h-17h, 9h-11h, 14h-16h, 11h-12h.

  Combien de ces activités peuvent-elles être conciliées sur une seule journée ?
  
2. On propose d'utiliser une stratégie gloutone pour sélectionner des activités en commençant par le début de la journée: choisir l'activité dont l'heure de fin arrive le plus tôt (parmi celles dont l'heure de début est bien postérieure aux créneaux des activités déjà choisies).

  Appliquer cette stratégie à la situation précédente.

#### Exercice 7 (programmation)

On reprend le problème précédent, et on suppose que l'on a ```n``` activités numérotées de  ```0``` à ```n-1```. On a aussi deux tableaux ```début``` et ```fin``` de taille ```n```, tels que ```début[i]``` et ```fin[i]``` contiennent respectivement les heures de début et de fin de l'activité numéro ```i```.

1. Écrire une fonction ```prochaine(début, fin, heure)``` qui sélectionne parmi les activités dont l'heure de début n'est pas antérieure à ```heure``` celle s'arrêtant le plut tôt.

  La fonction renverra ```None``` s'il n'y a aucun créneau compatible.

In [ ]:
def prochaine(début, fin, heure):
    pass

2. En déduire une fonction ```sélection(début, fin)``` qui, en supposant que toutes les heures sont positives, sélectionne autant d'activités que possible en suivant la stratégie gloutonne. On demandera à la fonction d'afficher les numéros des activités sélectionnées.

In [ ]:
def sélection(début, fin):
    pass